### 추천시스템 - Content based filtering
##### ref. https://lsjsj92.tistory.com/565

In [43]:
import pandas as pd
# 사용데이터 - 캐글 영화 데이터
data = pd.read_csv('./movie_data/movies_metadata.csv')
data.head(5) #확인용

/var/folders/35/3dnq9h1x55d5hz_q1748w2040000gn/T/ipykernel_61801/32832120.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./movie_data/movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [44]:
# 데이터 크기 확인
data.shape

(45466, 24)

In [45]:
# 사용할 column 정리
data = data[['id', 'genres', 'vote_average', 'vote_count', 'popularity', 'title', 'overview']]
# imdb에서 평점 처리를 위해 사용한 방법 weighted rating 사용
m = data['vote_count'].quantile(0.98) #상위 0.98로 설정 -> 값 확인하면서 변경
using_data = data.loc[data['vote_count'] >= m]

In [46]:
# 사용데이터 크기 확인 (영화개수, 사용column)
using_data.shape

(910, 7)

In [47]:
C = using_data['vote_average'].mean()
def weighted_rating(x, m=m, C=C):
	v = x['vote_count']
	R = x['vote_average']
	return (v/(v+m)*R) + (m/(m+v)*C)

In [48]:
# 새로 정의한 점수 적용
using_data['score']= using_data.apply(weighted_rating, axis=1)
using_data.head() #확인

/var/folders/35/3dnq9h1x55d5hz_q1748w2040000gn/T/ipykernel_61801/2280684382.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  using_data['score']= using_data.apply(weighted_rating, axis=1)


,id,genres,vote_average,vote_count,popularity,title,overview,score
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",7.7,5415.0,21.946943,Toy Story,"Led by Woody, Andy's toys live happily in his ...",7.536723
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9,2413.0,17.015539,Jumanji,When siblings Judy and Peter discover an encha...,6.873523
5,949,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",7.7,1886.0,17.924927,Heat,"Obsessive master thief, Neil McCauley leads a ...",7.352208
15,524,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",7.8,1343.0,10.137389,Casino,The life of the gambling paradise – Las Vegas ...,7.331063
31,63,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",7.4,2470.0,12.297305,Twelve Monkeys,"In the year 2035, convict James Cole reluctant...",7.207100


In [49]:
# 유사도 계산을 위해 사용할 장르 컬럼만 확인
using_data['genres'].head()

0     [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1     [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
5     [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...
15    [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...
31    [{'id': 878, 'name': 'Science Fiction'}, {'id'...
Name: genres, dtype: object

In [50]:
from ast import literal_eval

# string 형태를 dictionary 로 변환
using_data['genres']= using_data['genres'].apply(literal_eval)

/var/folders/35/3dnq9h1x55d5hz_q1748w2040000gn/T/ipykernel_61801/3101927120.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  using_data['genres']= using_data['genres'].apply(literal_eval)


In [51]:
# 장르 컬럼에서 id는 필요 없기 때문에 삭제
using_data['genres'] = using_data['genres'].apply(lambda x: [d['name'] for d in x]).apply(lambda x: " ".join(x))

using_data.head(2)

/var/folders/35/3dnq9h1x55d5hz_q1748w2040000gn/T/ipykernel_61801/311840894.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  using_data['genres'] = using_data['genres'].apply(lambda x: [d['name'] for d in x]).apply(lambda x: " ".join(x))


,id,genres,vote_average,vote_count,popularity,title,overview,score
0,862,Animation Comedy Family,7.7,5415.0,21.946943,Toy Story,"Led by Woody, Andy's toys live happily in his ...",7.536723
1,8844,Adventure Fantasy Family,6.9,2413.0,17.015539,Jumanji,When siblings Judy and Peter discover an encha...,6.873523


In [52]:
from sklearn.feature_extraction.text import CountVectorizer

# 문자열 -> 숫자 벡터화
count_vector = CountVectorizer(ngram_range=(1,3))
c_vector_genres = count_vector.fit_transform(using_data['genres'])

c_vector_genres.shape

(910, 550)

In [53]:
from sklearn.metrics.pairwise import cosine_similarity
# 코사인 유사도 계산 -> 유사도가 가장 높은 인덱스를 먼저 정렬
genre_c_sim = cosine_similarity(c_vector_genres, c_vector_genres).argsort()[:,::-1]

In [54]:
# 입력된 영화와 비슷한 장르의 영화를 리턴
def get_recommend_movie_list(df, input_movie_title, top=30):
	# 데이터에 입력된 영화 정보 추출
	target_movie_index = df[df['title'] == input_movie_title].index.values
	# 코사인 유사도를 계산하여 상위 30(default)의 인덱스 저장
	sim_index = genre_c_sim[target_movie_index, :top].reshape(-1)
	sim_index = sim_index[sim_index != target_movie_index] # 입력된 영화 제외
	result = df.iloc[sim_index].sort_values('score', ascending=False)[:10] #score순서로 정렬후 출력
	return result

In [55]:
get_recommend_movie_list(using_data, input_movie_title='Toy Story')

,id,genres,vote_average,vote_count,popularity,title,overview,score
13724,14160,Animation Comedy Family Adventure,7.8,7048.0,19.330884,Up,Carl Fredricksen spent his entire life dreamin...,7.653977
4756,585,Animation Comedy Family,7.5,6150.0,26.419962,"Monsters, Inc.","James Sullivan and Mike Wazowski are monsters,...",7.386456
11567,2062,Animation Comedy Family Fantasy,7.5,4510.0,20.50803,Ratatouille,A rat named Remy dreams of becoming a great Fr...,7.354054
22718,137106,Adventure Animation Comedy Family Fantasy,7.5,3127.0,16.418133,The Lego Movie,"An ordinary Lego mini-figure, mistakenly thoug...",7.307800
4178,808,Adventure Animation Comedy Family Fantasy,7.3,4183.0,17.987728,Shrek,It ain't easy bein' green -- especially if you...,7.190889
2997,863,Animation Comedy Family,7.3,3914.0,17.547693,Toy Story 2,"Andy heads off to Cowboy Camp, leaving his toy...",7.185191
5084,425,Animation Comedy Family Adventure,7.1,3954.0,17.328902,Ice Age,"With the impending ice age almost upon them, a...",7.033729
3891,11688,Adventure Animation Comedy Family Fantasy,7.2,1544.0,12.297375,The Emperor's New Groove,Kuzco is a self-centered emperor who summons P...,7.031819
41976,324849,Action Animation Comedy Family Fantasy,7.2,1473.0,17.070748,The Lego Batman Movie,In the irreverent spirit of fun that made “The...,7.027413
608,10112,Animation Comedy Family Adventure,7.1,1287.0,9.834047,The Aristocats,When Madame Adelaide Bonfamille leaves her for...,6.963699
